In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getPathList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=getPathList(directory_path)

# テキストファイルをデータフレームに格納する
df = fileToDataFrame(file_list)

In [3]:
from preProcess import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.08314013481140137 s


In [5]:
# トリプルテーブルを作る

from tripleExtraction import tripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

counter = 0
for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(triples))
    triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(triples))
    triple_dict['Triple'].extend(triples)
    
triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')
        
end = time.time()
print(end-start,'s')

393.9632821083069 s


In [6]:
'''
output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')
output.to_csv('output/allen_p_inbox_triple2.csv')

output = pd.read_csv('output/allen_p_inbox_triple2.csv')
'''

In [6]:
sentence_df.head()

,Message-ID,Sentence-ID,Sentence
0,<22534132.1075858645229.JavaMail.evans@thyme>,0,"=20 Updated: Oct. 30, 2001 TVA takes lead in sales, generation by utilities Quasi-governmental ..."
1,<22534132.1075858645229.JavaMail.evans@thyme>,1,Coalition aims to sell residential green power in R.I. full story... Enron's debt ratings cut fu...
2,<22534132.1075858645229.JavaMail.evans@thyme>,2,Domestic pipeline capacity must grow to meet expected gas demand full story... CMS could expand ...
3,<22534132.1075858645229.JavaMail.evans@thyme>,3,"2001 - Platts, All Rights Reserved Market Brief Monday, October 29 Stocks Close Change % Change ..."
4,<22534132.1075858645229.JavaMail.evans@thyme>,4,"296.10 (0.6) -0.22% NASDAQ 1,699.52 (69.44) -3.93% S&P 500 1,077.48 (26.3) -2.38% Market Vols Cl..."


In [4]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")
result = predictor.predict(sentence = "Did Uriah honestly think he could beat The Legend of Zelda in under three hours?")

/Users/taroaso/.pyenv/versions/anaconda3-2019.07/envs/allennlp/lib/python3.7/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:61: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning,


In [10]:
for i, tags in enumerate(result['tags']):
    if tags != 'O':
        print(result['words'][i])

CMS
LNG
Executive
News


In [16]:
result = predictor.predict(sentence = sentence_df['Sentence'][3])

In [13]:
result

{'logits': [[12.955562591552734,
   -3.3144965171813965,
   -2.2392969131469727,
   -5.791905403137207,
   -2.9718408584594727,
   -4.552072525024414,
   -1.3463222980499268,
   -2.315471649169922,
   -2.328221321105957,
   -1.5843279361724854,
   -4.878124713897705,
   -4.315371036529541,
   -2.7178938388824463,
   -3.406167507171631,
   -2.2008440494537354,
   -4.984387397766113,
   -3.8245701789855957],
  [13.971536636352539,
   -4.1494293212890625,
   -2.5604915618896484,
   -4.921985149383545,
   -3.928619384765625,
   -4.9945478439331055,
   -2.477180242538452,
   -1.5639359951019287,
   -2.9275803565979004,
   -0.9619989395141602,
   -5.856423854827881,
   -3.9780356884002686,
   -3.369739532470703,
   -2.4048500061035156,
   -1.4735705852508545,
   -4.361250400543213,
   -3.029815196990967],
  [13.742960929870605,
   -4.058728218078613,
   -2.3451499938964844,
   -4.390352249145508,
   -4.376400947570801,
   -4.431001663208008,
   -2.8492159843444824,
   -2.0428805351257324,
  

In [12]:
'''
allennlpのNERを試す
for sentence in sentence_df['Sentence']:
    result = predictor.predict(sentence = sentence)
    for i, tags in enumerate(result['tags']):
        if tags != 'O':
            print(result['words'][i])
時間がかかるし， spacyで十分
'''

TVA
Quasi
Tennessee
Valley
Authority
TVA
Boulder
Colo.-based
RDI
POWERdat
Mexican
ASP
ASPs
SPR
Nabors
Command
Drilling
Georgia
Power
Nigerian
R.I.
Enron
Dabhol
Mexico
NERC
Boston
LNG
CMS
LNG
Executive
News
Platts
All
Rights
Reserved
Util
NASDAQ
S&P
NASDAQ
Palo
Verde
Australia
Canada
Germany
Dmark
Euro
Japan
Mexico
UK
Arg
MerVal
Austr
All
Ord
Braz
Bovespa
TSE
Germany
DAX
HK
HangSeng
Japan
Nikkei
Mexico
IPC
UK
FTSE
Yahoo
!
Hayden
Frank
Sent
David
Subject
Gas
P&L
Jones
Brad
Sent
Allen
Phillip
K.
Hayden
Frank
Gossett
Jeffrey
C.
Gas
P&L
Brad
Jones
PHILLIP
America
Internet
Hayo
Portable
CD
Player
Motorola
Talkabout
Two
Way
Radios
Pen
Cam
Trio
Sprint
AnyTimeSM
Online
Sprint
Sprint
Sprint
New
Year
NETCO
Re
Louise
These
Faith
Killen
First
Tammy
Shepperd
Enron
South
Tammy
Shepperd
Andy
Zipper
John
Netco
Jeanie
Slone
New
Year
Netco
David
Oxley
New
Gen
Weekly
James
Bruce
Enron
North
America
West
Power
Desk
SW
Salmon
Portland
OR
DAVID
COURSEY
Sylvia
Carr
BIZ
HP
Peer
Napster
P2P
Wayne
Rash
IT
Wayne


KeyboardInterrupt: 

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")
for sentence in sentence_df['Sentence']:
    doc = nlp(sentence)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

20 2 4 CARDINAL
Oct. 30, 2001 14 27 DATE
TVA 28 31 ORG
Tennessee Valley Authority 103 129 ORG
TVA 131 134 ORG
2000 223 227 DATE
Boulder 263 270 GPE
RDI POWERdat 284 296 ORG
Mexican 315 322 NORP
Dependence 408 418 GPE
Utilities 449 458 ORG
ASP 476 479 ORG
Boosting 560 568 ORG
the SPR Calls 585 598 ORG
Nabors 698 704 ORG
Command Drilling 726 742 ORG
Georgia Power 757 770 ORG
Nigerian 798 806 NORP
R.I. 50 54 GPE
Enron 69 74 ORG
Dabhol 136 142 GPE
Mexico 165 171 GPE
next decade 270 281 DATE
Boston 316 322 GPE
LNG 340 343 ORG
CMS 79 82 ORG
LNG 96 99 ORG
400,000 112 119 CARDINAL
today 157 162 DATE
Executive News 165 179 ORG
2001 - Platts 0 13 DATE
All Rights Reserved Market Brief 15 47 WORK_OF_ART
Monday, October 29 48 66 DATE
Stocks Close Change % Change 67 95 ORG
9,269.50 101 109 CARDINAL
275.7 111 116 CARDINAL
-2.89% 118 124 PERCENT
DJ 15 125 130 LAW
296.10 0 6 CARDINAL
-0.22% 13 19 PERCENT
NASDAQ 1,699.52 20 35 ORG
69.44 37 42 CARDINAL
-3.93% 44 50 PERCENT
1,077.48 59 67 DATE
26.3 69 73 